In [1]:
import pandas as pd
import os
import numpy
import datetime
import re
import yfinance as yf

In [2]:
senate_inv = pd.read_csv('/Users/paigeblackstone/Desktop/Portfolio29/Portfolio29/Investment_Management/UpdatedSenatorTrades.csv')
senate_inv2 = pd.read_csv('/Users/paigeblackstone/Desktop/Portfolio29/Portfolio29/Investment_Management/senate_stock_discosures.csv')
senate_inv3 = pd.read_csv('/Users/paigeblackstone/Desktop/Portfolio29/Portfolio29/Investment_Management/SenatorCleaned.csv')

In [3]:
encodings = ['latin1', 'ISO-8859-1', 'cp1252']
for encoding in encodings:
    try:
        congress_inv = pd.read_csv('/Users/paigeblackstone/Desktop/Portfolio29/Portfolio29/Investment_Management/Copy of congress-trading-all (3).csv', encoding=encoding)
        print(f'Successfully read the file with encoding: {encoding}')
        break
    except UnicodeDecodeError:
        print(f'Failed to read the file with encoding: {encoding}')

Successfully read the file with encoding: latin1


In [4]:
congress_inv.columns.to_list()

['Ticker',
 'TickerType',
 'Company',
 'Traded',
 'Transaction',
 'Trade_Size_USD',
 'Status',
 'Subholding',
 'Description',
 'Name',
 'Filed',
 'Party',
 'District',
 'Chamber',
 'Comments',
 'Quiver_Upload_Time',
 'excess_return',
 'State',
 'last_modified']

cleaning up date columns

In [5]:
congress_inv['Traded'] = pd.to_datetime(congress_inv['Traded'], format='%A, %B %d, %Y')

# Display the updated DataFrame and check the data types
print(congress_inv)
print(congress_inv.dtypes)

                       Ticker TickerType  \
0                         NGL         ST   
1                         FCX         ST   
2                           V         ST   
3      APPLE INC. (XNAS:AAPL)         ST   
4                        MSFT         ST   
...                       ...        ...   
46462                     CAT        NaN   
46463                      PG        NaN   
46464                     KSU        NaN   
46465                    EP$C        NaN   
46466                    EP$C        NaN   

                                                 Company     Traded  \
0      NGL ENERGY PARTNERS LP COMMON UNITS REPRESENTI... 2024-03-11   
1                    FREEPORT-MCMORAN, INC. COMMON STOCK 2024-02-29   
2                                              VISA INC. 2024-02-29   
3                              APPLE INC. - COMMON STOCK 2024-02-29   
4                   MICROSOFT CORPORATION - COMMON STOCK 2024-02-29   
...                                          

In [6]:
congress_inv['Filed'] = pd.to_datetime(congress_inv['Filed'], errors='coerce')

# Check for any conversion issues
print(congress_inv['Filed'].head())

0   2024-03-13
1   2024-03-07
2   2024-03-07
3   2024-02-29
4   2024-03-08
Name: Filed, dtype: datetime64[ns]


Cleaning up tickers data and replacing with a mapping of cleaned tickers

In [7]:
ticker_list = congress_inv['Ticker'].unique().tolist()
print(ticker_list)
unique_tickers = ticker_list

['NGL', 'FCX', 'V', 'APPLE INC. (XNAS:AAPL)', 'MSFT', 'BA', 'URI', 'TDG', 'DUK$A', 'ZTS', 'PFE', 'QCOM', 'SBLK', 'ASC', 'SSBK', 'GOGL', 'FLNG', 'LRLCY', 'ARLP', 'BBY', 'SBUX', 'AMD', 'PANW', 'ET', 'ACN', 'NVDA', 'AMAT', 'TGT', 'GLA', 'EQNR', 'PG', 'MMC', 'NWN', 'BDX', 'OKE', 'TEX', 'TXN', 'DBSDY', 'CHRD', 'ILMN', 'HUMA', 'PHG', 'KEY', 'AMZN', 'CTSH', 'UL', 'TSM', 'BKNG', 'MDT', 'REGN', 'MDB', 'META', 'KKR', 'LLY', 'NFLX', 'GLAS FUNDS, LP', 'HAMILTO', 'T$A', 'AFRM', 'SPG', 'LNG', 'TTD', 'DE', 'IBM', 'MRNA', 'FSLR', 'SCHW', 'MS', 'MSCI', 'GOOGL', 'EXAS', 'HTGC', 'COST', 'JPM', 'PRNDY', 'ALNY', 'ISRG', 'VIG', 'USFR', 'GOOG', 'BRK.B', 'LRCX', 'NOW', 'TSLA', 'AAPL', 'ON', 'APH', 'ABG', 'TPH', 'LOW', 'AGIO', 'INTC', 'RPM', 'MEDP', 'LAD', 'MRK', 'SYY', '4.WEEK, MATURE', 'AMGN', 'INTU', 'TJX', 'AON', 'CFG', 'CAT', 'MU', 'CME', 'NKE', 'ABT', 'CRM', 'CB', 'PKG', 'ABBV', 'MA', 'LMT', 'WMT', 'JNJ', 'GIS', 'SCCO', 'MNST', 'CLF', 'IDXX', 'FMAO', 'BABA', 'PAM', 'RTX', 'HQY', 'ADSK', 'DIS', 'NVO', 'CO

In [8]:
cleaned_tickers = []
for ticker in unique_tickers:

    match = re.search(r'\(([^)]+)\)', ticker)
    if match:
        ticker = match.group(1).split(":")[-1]  

    ticker = re.sub(r'[^A-Z0-9-]', '', ticker)

    if ticker:
        cleaned_tickers.append(ticker)

In [10]:
ticker_mapping = dict(zip(congress_inv['Ticker'], cleaned_tickers))
congress_inv['Ticker'] = congress_inv['Ticker'].map(ticker_mapping)
congress_inv = congress_inv[congress_inv['Ticker'].isin(cleaned_tickers)]
print(congress_inv)

      Ticker TickerType                                            Company  \
0        GMT         ST  NGL ENERGY PARTNERS LP COMMON UNITS REPRESENTI...   
1        DDR         ST                FREEPORT-MCMORAN, INC. COMMON STOCK   
2        BCR         ST                                          VISA INC.   
3       AAPL         ST                          APPLE INC. - COMMON STOCK   
4       RYAM         ST               MICROSOFT CORPORATION - COMMON STOCK   
...      ...        ...                                                ...   
46459   MNTA        NaN                        LOCKHEED MARTIN CORPORATION   
46460  VPMAX      Stock        E. I. du Pont de Nemours and Company (NYSE)   
46461   BETR        NaN                           PROCTER & GAMBLE COMPANY   
46462   SIMO        NaN                                  CATERPILLAR, INC.   
46463   BETR        NaN                           PROCTER & GAMBLE COMPANY   

          Traded Transaction       Trade_Size_USD Status  \
0  

observing counts of tickers traded by filed date and by traded date

In [11]:
ticker_counts_by_filed_date = congress_inv.groupby(['Filed', 'Ticker', 'Company', 'Transaction']).size().reset_index(name='Count')
print(ticker_counts_by_filed_date)

ticker_counts_by_traded_date = congress_inv.groupby(['Traded', 'Ticker', 'Company', 'Transaction']).size().reset_index(name='Count')
print(ticker_counts_by_traded_date)

           Filed     Ticker                                          Company  \
0     2014-01-07  36966R5P7                              TJX COMPANIES, INC.   
1     2014-01-07       ADTF                          MASTERCARD INCORPORATED   
2     2014-01-07       ATRO      INTERNATIONAL BUSINESS MACHINES CORPORATION   
3     2014-01-07        BWX                                       3M COMPANY   
4     2014-01-07        BYI                  PHILIP MORRIS INTERNATIONAL INC   
...          ...        ...                                              ...   
23907 2024-03-15        PCH  Southern States Bancshares, Inc. - Common Stock   
23908 2024-03-15       SBLK         Star Bulk Carriers Corp. - Common Shares   
23909 2024-03-15        SPB                                   Starbucks Corp   
23910 2024-03-15       SWAY                        Pfizer, Inc. Common Stock   
23911 2024-03-15         TV             QUALCOMM Incorporated - Common Stock   

       Transaction  Count  
0         P

In [12]:
monthly_counts_by_filed_date = congress_inv.groupby([pd.Grouper(key='Filed', freq='M'), 'Ticker']).size().reset_index(name='Count')
monthly_counts_by_filed_date['Filed'] = monthly_counts_by_filed_date['Filed'].dt.to_period('M').astype(str)

monthly_counts_by_traded_date = congress_inv.groupby([pd.Grouper(key='Traded', freq='M'), 'Ticker']).size().reset_index(name='Count')
monthly_counts_by_traded_date['Filed'] = monthly_counts_by_traded_date['Traded'].dt.to_period('M').astype(str)

average length of time between trade date and file date by name, by ticker, and when transaction= Purchase and when transcation= Sale

In [14]:
congress_inv['Traded'] = pd.to_datetime(congress_inv['Traded'], format='%A, %B %d, %Y')

congress_inv['Filed'] = pd.to_datetime(congress_inv['Filed'], format='%Y-%m-%d')

In [15]:
congress_inv['days_traded_to_filed'] = (congress_inv['Filed'] - congress_inv['Traded']).dt.days
print(congress_inv[['Traded', 'Filed', 'days_traded_to_filed']])

          Traded      Filed  days_traded_to_filed
0     2024-03-11 2024-03-13                     2
1     2024-02-29 2024-03-07                     7
2     2024-02-29 2024-03-07                     7
3     2024-02-29 2024-02-29                     0
4     2024-02-29 2024-03-08                     8
...          ...        ...                   ...
46459 2012-11-13 2014-07-08                   602
46460 2012-09-13 2015-05-13                   972
46461 2012-08-16 2014-07-08                   691
46462 2012-07-26 2014-07-08                   712
46463 2012-07-24 2014-07-08                   714

[29862 rows x 3 columns]


collecting historical stock data based on tickers in the congress financial disclosures dataframe, starting in 2022

In [19]:
import yfinance as yf
import pandas as pd

def get_stock_info(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        if not info:  
            raise ValueError("No data found for ticker")
        return {
                    'Ticker': ticker,
                    'Industry': info.get('industry', 'N/A'),
                    'Sector': info.get('sector', 'N/A'),
                    'Market Cap': info.get('marketCap', 'N/A'),
                    'PE Ratio': info.get('trailingPE', 'N/A'),
                    'Price to Book': info.get('priceToBook', 'N/A'),
                    'Return on Equity': info.get('returnOnEquity', 'N/A'),
                    'Price to Sales': info.get('priceToSalesTrailing12Months', 'N/A'),
                    'Dividend Yield': info.get('dividendYield', 'N/A'),
                    'Current Ratio': info.get('currentRatio', 'N/A'),
                    'Beta': info.get('beta', 'N/A'),
                    'Address1': info.get('address1', 'N/A'),
                    'City': info.get('city', 'N/A'),
                    'State': info.get('state', 'N/A'),
                    'Zip': info.get('zip', 'N/A'),
                    'Country': info.get('country', 'N/A'),
                    'Phone': info.get('phone', 'N/A'),
                    'Website': info.get('website', 'N/A'),
                    'Industry Key': info.get('industryKey', 'N/A'),
                    'Industry Disp': info.get('industryDisp', 'N/A'),
                    'Sector Key': info.get('sectorKey', 'N/A'),
                    'Sector Disp': info.get('sectorDisp', 'N/A'),
                    'Long Business Summary': info.get('longBusinessSummary', 'N/A'),
                    'Full Time Employees': info.get('fullTimeEmployees', 'N/A'),
                    'Company Officers': info.get('companyOfficers', 'N/A'),
                    'Audit Risk': info.get('auditRisk', 'N/A'),
                    'Board Risk': info.get('boardRisk', 'N/A'),
                    'Compensation Risk': info.get('compensationRisk', 'N/A'),
                    'Shareholder Rights Risk': info.get('shareHolderRightsRisk', 'N/A'),
                    'Overall Risk': info.get('overallRisk', 'N/A'),
                    'Governance Epoch Date': info.get('governanceEpochDate', 'N/A'),
                    'Compensation As Of Epoch Date': info.get('compensationAsOfEpochDate', 'N/A'),
                    'IR Website': info.get('irWebsite', 'N/A'),
                    'Max Age': info.get('maxAge', 'N/A'),
                    'Price Hint': info.get('priceHint', 'N/A'),
                    'Previous Close': info.get('previousClose', 'N/A'),
                    'Open': info.get('open', 'N/A'),
                    'Day Low': info.get('dayLow', 'N/A'),
                    'Day High': info.get('dayHigh', 'N/A'),
                    'Regular Market Previous Close': info.get('regularMarketPreviousClose', 'N/A'),
                    'Regular Market Open': info.get('regularMarketOpen', 'N/A'),
                    'Regular Market Day Low': info.get('regularMarketDayLow', 'N/A'),
                    'Regular Market Day High': info.get('regularMarketDayHigh', 'N/A'),
                    'Dividend Rate': info.get('dividendRate', 'N/A'),
                    'Ex Dividend Date': info.get('exDividendDate', 'N/A'),
                    'Payout Ratio': info.get('payoutRatio', 'N/A'),
                    'Five Year Avg Dividend Yield': info.get('fiveYearAvgDividendYield', 'N/A'),
                    'Forward PE': info.get('forwardPE', 'N/A'),
                    'Volume': info.get('volume', 'N/A'),
                    'Regular Market Volume': info.get('regularMarketVolume', 'N/A'),
                    'Average Volume': info.get('averageVolume', 'N/A'),
                    'Average Volume 10 Days': info.get('averageVolume10days', 'N/A'),
                    'Bid': info.get('bid', 'N/A'),
                    'Ask': info.get('ask', 'N/A'),
                    'Bid Size': info.get('bidSize', 'N/A'),
                    'Ask Size': info.get('askSize', 'N/A'),
                    'Fifty Two Week Low': info.get('fiftyTwoWeekLow', 'N/A'),
                    'Fifty Two Week High': info.get('fiftyTwoWeekHigh', 'N/A'),
                    'Price To Sales Trailing 12 Months': info.get('priceToSalesTrailing12Months', 'N/A'),
                    'Fifty Day Average': info.get('fiftyDayAverage', 'N/A'),
                    'Two Hundred Day Average': info.get('twoHundredDayAverage', 'N/A'),
                    'Trailing Annual Dividend Rate': info.get('trailingAnnualDividendRate', 'N/A'),
                    'Trailing Annual Dividend Yield': info.get('trailingAnnualDividendYield', 'N/A'),
                    'Currency': info.get('currency', 'N/A'),
                    'Enterprise Value': info.get('enterpriseValue', 'N/A'),
                    'Profit Margins': info.get('profitMargins', 'N/A'),
                    'Float Shares': info.get('floatShares', 'N/A'),
                    'Shares Outstanding': info.get('sharesOutstanding', 'N/A'),
                    'Shares Short': info.get('sharesShort', 'N/A'),
                    'Shares Short Prior Month': info.get('sharesShortPriorMonth', 'N/A'),
                    'Shares Short Previous Month Date': info.get('sharesShortPreviousMonthDate', 'N/A'),
                    'Date Short Interest': info.get('dateShortInterest', 'N/A'),
                    'Shares Percent Shares Out': info.get('sharesPercentSharesOut', 'N/A'),
                    'Held Percent Insiders': info.get('heldPercentInsiders', 'N/A'),
                    'Held Percent Institutions': info.get('heldPercentInstitutions', 'N/A'),
                    'Short Ratio': info.get('shortRatio', 'N/A'),
                    'Short Percent Of Float': info.get('shortPercentOfFloat', 'N/A'),
                    'Implied Shares Outstanding': info.get('impliedSharesOutstanding', 'N/A'),
                    'Book Value': info.get('bookValue', 'N/A'),
                    'Last Fiscal Year End': info.get('lastFiscalYearEnd', 'N/A'),
                    'Next Fiscal Year End': info.get('nextFiscalYearEnd', 'N/A'),
                    'Most Recent Quarter': info.get('mostRecentQuarter', 'N/A'),
                    'Earnings Quarterly Growth': info.get('earningsQuarterlyGrowth', 'N/A'),
                    'Net Income To Common': info.get('netIncomeToCommon', 'N/A'),
                    'Trailing EPS': info.get('trailingEps', 'N/A'),
                    'Forward EPS': info.get('forwardEps', 'N/A'),
                    'PEG Ratio': info.get('pegRatio', 'N/A'),
                    'Last Split Factor': info.get('lastSplitFactor', 'N/A'),
                    'Last Split Date': info.get('lastSplitDate', 'N/A'),
                    'Enterprise To Revenue': info.get('enterpriseToRevenue', 'N/A'),
                    'Enterprise To EBITDA': info.get('enterpriseToEbitda', 'N/A'),
                    '52 Week Change': info.get('52WeekChange', 'N/A'),
                    'SandP 52 Week Change': info.get('SandP52WeekChange', 'N/A'),
                    'Last Dividend Value': info.get('lastDividendValue', 'N/A'),
                    'Last Dividend Date': info.get('lastDividendDate', 'N/A'),
                    'Exchange': info.get('exchange', 'N/A'),
                    'Quote Type': info.get('quoteType', 'N/A'),
                    'Symbol': info.get('symbol', 'N/A'),
                    'Underlying Symbol': info.get('underlyingSymbol', 'N/A'),
                    'Short Name': info.get('shortName', 'N/A'),
                    'Long Name': info.get('longName', 'N/A'),
                    'First Trade Date Epoch UTC': info.get('firstTradeDateEpochUtc', 'N/A'),
                    'Time Zone Full Name': info.get('timeZoneFullName', 'N/A'),
                    'Time Zone Short Name': info.get('timeZoneShortName', 'N/A'),
                    'UUID': info.get('uuid', 'N/A'),
                    'Message Board ID': info.get('messageBoardId', 'N/A'),
                    'GMT Offset Milliseconds': info.get('gmtOffSetMilliseconds', 'N/A'),
                    'Current Price': info.get('currentPrice', 'N/A'),
                    'Target High Price': info.get('targetHighPrice', 'N/A'),
                    'Target Low Price': info.get('targetLowPrice', 'N/A'),
                    'Target Mean Price': info.get('targetMeanPrice', 'N/A'),
                    'Target Median Price': info.get('targetMedianPrice', 'N/A'),
                    'Recommendation Mean': info.get('recommendationMean', 'N/A'),
                    'Recommendation Key': info.get('recommendationKey', 'N/A'),
                    'Number Of Analyst Opinions': info.get('numberOfAnalystOpinions', 'N/A'),
                    'Total Cash': info.get('totalCash', 'N/A'),
                    'Total Cash Per Share': info.get('totalCashPerShare', 'N/A'),
                    'EBITDA': info.get('ebitda', 'N/A'),
                    'Total Debt': info.get('totalDebt', 'N/A'),
                    'Quick Ratio': info.get('quickRatio', 'N/A'),
                    'Total Revenue': info.get('totalRevenue', 'N/A'),
                    'Debt To Equity': info.get('debtToEquity', 'N/A'),
                    'Revenue Per Share': info.get('revenuePerShare', 'N/A'),
                    'Return on Assets': info.get('returnOnAssets', 'N/A'),
                    'Return on Equity': info.get('returnOnEquity', 'N/A'),
                    'Free Cash Flow': info.get('freeCashflow', 'N/A'),
                    'Operating Cash Flow': info.get('operatingCashflow', 'N/A'),
                    'Earnings Growth': info.get('earningsGrowth', 'N/A'),
                    'Revenue Growth': info.get('revenueGrowth', 'N/A'),
                    'Gross Margins': info.get('grossMargins', 'N/A'),
                    'EBITDA Margins': info.get('ebitdaMargins', 'N/A'),
                    'Operating Margins': info.get('operatingMargins', 'N/A'),
                    'Financial Currency': info.get('financialCurrency', 'N/A'),
                    'Trailing PEG Ratio': info.get('trailingPegRatio', 'N/A')}
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

def get_monthly_data(ticker, start_date):
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date)
        if hist.empty:
            raise ValueError("No historical data found for ticker")

        hist_resampled = hist.resample('M').agg({
            'Open': 'mean',
            'High': 'max',
            'Low': 'min',
            'Close': 'mean',
            'Volume': 'sum'
        })
        expected_keys = ['Ticker', 'Industry', 'Sector', 'Market Cap', 'PE Ratio', 'Price to Book', 'Return on Equity',
                        'Price to Sales', 'Dividend Yield', 'Current Ratio', 'Beta', 'Address1', 'City', 'State', 'Zip',
                        'Country', 'Phone', 'Website', 'Industry Key', 'Industry Disp', 'Sector Key', 'Sector Disp',
                        'Long Business Summary', 'Full Time Employees', 'Company Officers', 'Audit Risk', 'Board Risk',
                        'Compensation Risk', 'Shareholder Rights Risk', 'Overall Risk', 'Governance Epoch Date',
                        'Compensation As Of Epoch Date', 'IR Website', 'Max Age', 'Price Hint', 'Previous Close', 'Open',
                        'Day Low', 'Day High', 'Regular Market Previous Close', 'Regular Market Open',
                        'Regular Market Day Low', 'Regular Market Day High', 'Dividend Rate', 'Ex Dividend Date',
                        'Payout Ratio', 'Five Year Avg Dividend Yield', 'Forward PE', 'Volume', 'Regular Market Volume',
                        'Average Volume', 'Average Volume 10 Days', 'Bid', 'Ask', 'Bid Size', 'Ask Size',
                        'Fifty Two Week Low', 'Fifty Two Week High', 'Price To Sales Trailing 12 Months',
                        'Fifty Day Average', 'Two Hundred Day Average', 'Trailing Annual Dividend Rate',
                        'Trailing Annual Dividend Yield', 'Currency', 'Enterprise Value', 'Profit Margins', 'Float Shares',
                        'Shares Outstanding', 'Shares Short', 'Shares Short Prior Month', 'Shares Short Previous Month Date',
                        'Date Short Interest', 'Shares Percent Shares Out', 'Held Percent Insiders', 'Held Percent Institutions',
                        'Short Ratio', 'Short Percent Of Float', 'Implied Shares Outstanding', 'Book Value',
                        'Last Fiscal Year End', 'Next Fiscal Year End', 'Most Recent Quarter', 'Earnings Quarterly Growth',
                        'Net Income To Common', 'Trailing EPS', 'Forward EPS', 'PEG Ratio', 'Last Split Factor', 'Last Split Date',
                        'Enterprise To Revenue', 'Enterprise To EBITDA', '52 Week Change', 'SandP 52 Week Change', 'Last Dividend Value',
                        'Last Dividend Date', 'Exchange', 'Quote Type', 'Symbol', 'Underlying Symbol', 'Short Name', 'Long Name',
                        'First Trade Date Epoch UTC', 'Time Zone Full Name', 'Time Zone Short Name', 'UUID', 'Message Board ID',
                        'GMT Offset Milliseconds', 'Current Price', 'Target High Price', 'Target Low Price', 'Target Mean Price',
                        'Target Median Price', 'Recommendation Mean', 'Recommendation Key', 'Number Of Analyst Opinions',
                        'Total Cash', 'Total Cash Per Share', 'EBITDA', 'Total Debt', 'Quick Ratio', 'Total Revenue', 'Debt To Equity',
                        'Revenue Per Share', 'Return on Assets', 'Return on Equity', 'Free Cash Flow', 'Operating Cash Flow',
                        'Earnings Growth', 'Revenue Growth', 'Gross Margins', 'EBITDA Margins', 'Operating Margins', 'Financial Currency',
                        'Trailing PEG Ratio']

        fundamentals = pd.DataFrame(index=hist_resampled.index,columns=expected_keys)

        for date in fundamentals.index:
            stock_info = get_stock_info(ticker)
            if stock_info:
                for key, value in stock_info.items():
                    fundamentals.at[date, key] = value
        monthly_data = pd.concat([hist_resampled, fundamentals], axis=1)
        monthly_data['Ticker'] = ticker

        return monthly_data

    except Exception as e:
        print(f"Error fetching historical data for {ticker}: {e}")
        return pd.DataFrame()

start_date = '2020-01-01'
for ticker in cleaned_tickers:
    monthly_data = get_monthly_data(ticker, start_date)
    if not monthly_data.empty:
        monthly_data.to_csv(f'/Users/paigeblackstone/Desktop/Portfolio29/Portfolio29/Investment_Management/datapull_YF_082724/{ticker}_monthly_data.csv')
        print(f"Monthly data for {ticker} saved successfully.")

print("Data collection completed.")


Monthly data for NGL saved successfully.
Monthly data for FCX saved successfully.
Monthly data for V saved successfully.
Monthly data for AAPL saved successfully.
Monthly data for MSFT saved successfully.
Monthly data for BA saved successfully.
Monthly data for URI saved successfully.
Monthly data for TDG saved successfully.


$DUKA: possibly delisted; No timezone found


Error fetching historical data for DUKA: No historical data found for ticker
Monthly data for ZTS saved successfully.
Monthly data for PFE saved successfully.
Monthly data for QCOM saved successfully.
Monthly data for SBLK saved successfully.
Monthly data for ASC saved successfully.
Monthly data for SSBK saved successfully.
Monthly data for GOGL saved successfully.


In [16]:
import yfinance as yf
import pandas as pd

def get_stock_info(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        if not info:  # Check if info is empty
            raise ValueError("No data found for ticker")
        return {
                    'Ticker': ticker,
                    'Industry': info.get('industry', 'N/A'),
                    'Sector': info.get('sector', 'N/A'),
                    'Market Cap': info.get('marketCap', 'N/A'),
                    'PE Ratio': info.get('trailingPE', 'N/A'),
                    'Price to Book': info.get('priceToBook', 'N/A'),
                    'Return on Equity': info.get('returnOnEquity', 'N/A'),
                    'Price to Sales': info.get('priceToSalesTrailing12Months', 'N/A'),
                    'Dividend Yield': info.get('dividendYield', 'N/A'),
                    'Current Ratio': info.get('currentRatio', 'N/A'),
                    'Beta': info.get('beta', 'N/A'),
                    'Address1': info.get('address1', 'N/A'),
                    'City': info.get('city', 'N/A'),
                    'State': info.get('state', 'N/A'),
                    'Zip': info.get('zip', 'N/A'),
                    'Country': info.get('country', 'N/A'),
                    'Phone': info.get('phone', 'N/A'),
                    'Website': info.get('website', 'N/A'),
                    'Industry Key': info.get('industryKey', 'N/A'),
                    'Industry Disp': info.get('industryDisp', 'N/A'),
                    'Sector Key': info.get('sectorKey', 'N/A'),
                    'Sector Disp': info.get('sectorDisp', 'N/A'),
                    'Long Business Summary': info.get('longBusinessSummary', 'N/A'),
                    'Full Time Employees': info.get('fullTimeEmployees', 'N/A'),
                    'Company Officers': info.get('companyOfficers', 'N/A'),
                    'Audit Risk': info.get('auditRisk', 'N/A'),
                    'Board Risk': info.get('boardRisk', 'N/A'),
                    'Compensation Risk': info.get('compensationRisk', 'N/A'),
                    'Shareholder Rights Risk': info.get('shareHolderRightsRisk', 'N/A'),
                    'Overall Risk': info.get('overallRisk', 'N/A'),
                    'Governance Epoch Date': info.get('governanceEpochDate', 'N/A'),
                    'Compensation As Of Epoch Date': info.get('compensationAsOfEpochDate', 'N/A'),
                    'IR Website': info.get('irWebsite', 'N/A'),
                    'Max Age': info.get('maxAge', 'N/A'),
                    'Price Hint': info.get('priceHint', 'N/A'),
                    'Previous Close': info.get('previousClose', 'N/A'),
                    'Open': info.get('open', 'N/A'),
                    'Day Low': info.get('dayLow', 'N/A'),
                    'Day High': info.get('dayHigh', 'N/A'),
                    'Regular Market Previous Close': info.get('regularMarketPreviousClose', 'N/A'),
                    'Regular Market Open': info.get('regularMarketOpen', 'N/A'),
                    'Regular Market Day Low': info.get('regularMarketDayLow', 'N/A'),
                    'Regular Market Day High': info.get('regularMarketDayHigh', 'N/A'),
                    'Dividend Rate': info.get('dividendRate', 'N/A'),
                    'Ex Dividend Date': info.get('exDividendDate', 'N/A'),
                    'Payout Ratio': info.get('payoutRatio', 'N/A'),
                    'Five Year Avg Dividend Yield': info.get('fiveYearAvgDividendYield', 'N/A'),
                    'Forward PE': info.get('forwardPE', 'N/A'),
                    'Volume': info.get('volume', 'N/A'),
                    'Regular Market Volume': info.get('regularMarketVolume', 'N/A'),
                    'Average Volume': info.get('averageVolume', 'N/A'),
                    'Average Volume 10 Days': info.get('averageVolume10days', 'N/A'),
                    'Bid': info.get('bid', 'N/A'),
                    'Ask': info.get('ask', 'N/A'),
                    'Bid Size': info.get('bidSize', 'N/A'),
                    'Ask Size': info.get('askSize', 'N/A'),
                    'Fifty Two Week Low': info.get('fiftyTwoWeekLow', 'N/A'),
                    'Fifty Two Week High': info.get('fiftyTwoWeekHigh', 'N/A'),
                    'Price To Sales Trailing 12 Months': info.get('priceToSalesTrailing12Months', 'N/A'),
                    'Fifty Day Average': info.get('fiftyDayAverage', 'N/A'),
                    'Two Hundred Day Average': info.get('twoHundredDayAverage', 'N/A'),
                    'Trailing Annual Dividend Rate': info.get('trailingAnnualDividendRate', 'N/A'),
                    'Trailing Annual Dividend Yield': info.get('trailingAnnualDividendYield', 'N/A'),
                    'Currency': info.get('currency', 'N/A'),
                    'Enterprise Value': info.get('enterpriseValue', 'N/A'),
                    'Profit Margins': info.get('profitMargins', 'N/A'),
                    'Float Shares': info.get('floatShares', 'N/A'),
                    'Shares Outstanding': info.get('sharesOutstanding', 'N/A'),
                    'Shares Short': info.get('sharesShort', 'N/A'),
                    'Shares Short Prior Month': info.get('sharesShortPriorMonth', 'N/A'),
                    'Shares Short Previous Month Date': info.get('sharesShortPreviousMonthDate', 'N/A'),
                    'Date Short Interest': info.get('dateShortInterest', 'N/A'),
                    'Shares Percent Shares Out': info.get('sharesPercentSharesOut', 'N/A'),
                    'Held Percent Insiders': info.get('heldPercentInsiders', 'N/A'),
                    'Held Percent Institutions': info.get('heldPercentInstitutions', 'N/A'),
                    'Short Ratio': info.get('shortRatio', 'N/A'),
                    'Short Percent Of Float': info.get('shortPercentOfFloat', 'N/A'),
                    'Implied Shares Outstanding': info.get('impliedSharesOutstanding', 'N/A'),
                    'Book Value': info.get('bookValue', 'N/A'),
                    'Last Fiscal Year End': info.get('lastFiscalYearEnd', 'N/A'),
                    'Next Fiscal Year End': info.get('nextFiscalYearEnd', 'N/A'),
                    'Most Recent Quarter': info.get('mostRecentQuarter', 'N/A'),
                    'Earnings Quarterly Growth': info.get('earningsQuarterlyGrowth', 'N/A'),
                    'Net Income To Common': info.get('netIncomeToCommon', 'N/A'),
                    'Trailing EPS': info.get('trailingEps', 'N/A'),
                    'Forward EPS': info.get('forwardEps', 'N/A'),
                    'PEG Ratio': info.get('pegRatio', 'N/A'),
                    'Last Split Factor': info.get('lastSplitFactor', 'N/A'),
                    'Last Split Date': info.get('lastSplitDate', 'N/A'),
                    'Enterprise To Revenue': info.get('enterpriseToRevenue', 'N/A'),
                    'Enterprise To EBITDA': info.get('enterpriseToEbitda', 'N/A'),
                    '52 Week Change': info.get('52WeekChange', 'N/A'),
                    'SandP 52 Week Change': info.get('SandP52WeekChange', 'N/A'),
                    'Last Dividend Value': info.get('lastDividendValue', 'N/A'),
                    'Last Dividend Date': info.get('lastDividendDate', 'N/A'),
                    'Exchange': info.get('exchange', 'N/A'),
                    'Quote Type': info.get('quoteType', 'N/A'),
                    'Symbol': info.get('symbol', 'N/A'),
                    'Underlying Symbol': info.get('underlyingSymbol', 'N/A'),
                    'Short Name': info.get('shortName', 'N/A'),
                    'Long Name': info.get('longName', 'N/A'),
                    'First Trade Date Epoch UTC': info.get('firstTradeDateEpochUtc', 'N/A'),
                    'Time Zone Full Name': info.get('timeZoneFullName', 'N/A'),
                    'Time Zone Short Name': info.get('timeZoneShortName', 'N/A'),
                    'UUID': info.get('uuid', 'N/A'),
                    'Message Board ID': info.get('messageBoardId', 'N/A'),
                    'GMT Offset Milliseconds': info.get('gmtOffSetMilliseconds', 'N/A'),
                    'Current Price': info.get('currentPrice', 'N/A'),
                    'Target High Price': info.get('targetHighPrice', 'N/A'),
                    'Target Low Price': info.get('targetLowPrice', 'N/A'),
                    'Target Mean Price': info.get('targetMeanPrice', 'N/A'),
                    'Target Median Price': info.get('targetMedianPrice', 'N/A'),
                    'Recommendation Mean': info.get('recommendationMean', 'N/A'),
                    'Recommendation Key': info.get('recommendationKey', 'N/A'),
                    'Number Of Analyst Opinions': info.get('numberOfAnalystOpinions', 'N/A'),
                    'Total Cash': info.get('totalCash', 'N/A'),
                    'Total Cash Per Share': info.get('totalCashPerShare', 'N/A'),
                    'EBITDA': info.get('ebitda', 'N/A'),
                    'Total Debt': info.get('totalDebt', 'N/A'),
                    'Quick Ratio': info.get('quickRatio', 'N/A'),
                    'Total Revenue': info.get('totalRevenue', 'N/A'),
                    'Debt To Equity': info.get('debtToEquity', 'N/A'),
                    'Revenue Per Share': info.get('revenuePerShare', 'N/A'),
                    'Return on Assets': info.get('returnOnAssets', 'N/A'),
                    'Return on Equity': info.get('returnOnEquity', 'N/A'),
                    'Free Cash Flow': info.get('freeCashflow', 'N/A'),
                    'Operating Cash Flow': info.get('operatingCashflow', 'N/A'),
                    'Earnings Growth': info.get('earningsGrowth', 'N/A'),
                    'Revenue Growth': info.get('revenueGrowth', 'N/A'),
                    'Gross Margins': info.get('grossMargins', 'N/A'),
                    'EBITDA Margins': info.get('ebitdaMargins', 'N/A'),
                    'Operating Margins': info.get('operatingMargins', 'N/A'),
                    'Financial Currency': info.get('financialCurrency', 'N/A'),
                    'Trailing PEG Ratio': info.get('trailingPegRatio', 'N/A')}
                                    

    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None 

def get_monthly_data(ticker, start_date):
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date)
        if hist.empty:
            raise ValueError("No historical data found for ticker")

        # Resample price data to monthly frequency
        hist_resampled = hist.resample('M').agg({
            'Open': 'mean',
            'High': 'max',
            'Low': 'min',
            'Close': 'mean',
            'Volume': 'sum'
        })

        fundamentals = pd.DataFrame(index=hist_resampled.index)

        for date in fundamentals.index:
            stock_info = get_stock_info(ticker)
            if stock_info:
                for key, value in stock_info.items():
                    fundamentals.at[date, key] = value

        monthly_data = pd.concat([hist_resampled, fundamentals], axis=1)
        monthly_data['Ticker'] = ticker

        return monthly_data

    except Exception as e:
        print(f"Error fetching historical data for {ticker}: {e}")
        return pd.DataFrame() 

start_date = '2020-01-01'
for ticker in unique_tickers:
    monthly_data = get_monthly_data(ticker, start_date)
    if not monthly_data.empty:
        monthly_data.to_csv(f'/Users/paigeblackstone/Desktop/Portfolio29/Portfolio29/Investment_Management/datapull_YF_081224/{ticker}_monthly_data.csv')
        print(f"Monthly data for {ticker} saved successfully.")

print("Data collection completed.")


Error fetching historical data for NGL: Must have equal len keys and value when setting with an iterable
Error fetching historical data for FCX: Must have equal len keys and value when setting with an iterable
Error fetching historical data for V: Must have equal len keys and value when setting with an iterable


$APPLE INC. (XNAS:AAPL): possibly delisted; No timezone found


Error fetching historical data for APPLE INC. (XNAS:AAPL): No historical data found for ticker
Error fetching historical data for MSFT: Must have equal len keys and value when setting with an iterable
Error fetching historical data for BA: Must have equal len keys and value when setting with an iterable
Error fetching historical data for URI: Must have equal len keys and value when setting with an iterable
Error fetching historical data for TDG: Must have equal len keys and value when setting with an iterable


$DUK$A: possibly delisted; No timezone found


Error fetching historical data for DUK$A: No historical data found for ticker
Error fetching historical data for ZTS: Must have equal len keys and value when setting with an iterable
Error fetching historical data for PFE: Must have equal len keys and value when setting with an iterable
Error fetching historical data for QCOM: Must have equal len keys and value when setting with an iterable
Error fetching historical data for SBLK: Must have equal len keys and value when setting with an iterable
Error fetching historical data for ASC: Must have equal len keys and value when setting with an iterable
Error fetching historical data for SSBK: Must have equal len keys and value when setting with an iterable
Error fetching historical data for GOGL: Must have equal len keys and value when setting with an iterable
Error fetching historical data for FLNG: Must have equal len keys and value when setting with an iterable
Error fetching historical data for LRLCY: Must have equal len keys and value 

...

$ETFC: possibly delisted; No timezone found


Error fetching historical data for ETFC: No historical data found for ticker


$BREW: possibly delisted; No timezone found


Error fetching historical data for BREW: No historical data found for ticker
Error fetching historical data for PBF: Must have equal len keys and value when setting with an iterable


KeyboardInterrupt: 

In [18]:
csv_folder = '/Users/paigeblackstone/Desktop/Portfolio29/Portfolio29/Investment_Management/datapull_YF_081224/'
dataframes = []

# Iterate through all CSV files in the folder
for filename in os.listdir(csv_folder):
    if filename.endswith('_monthly_data.csv'):
        # Extract the ticker from the filename
        ticker = filename.split('_')[0]
        
        # Load the CSV file into a DataFrame
        file_path = os.path.join(csv_folder, filename)
        df = pd.read_csv(file_path)
        
        # Add a column for the Ticker
        df['Ticker'] = ticker
        
        # Append the DataFrame to the list
        dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Display the combined DataFrame
print(combined_df)

                             Date       Open       High        Low      Close  \
0       2020-01-31 00:00:00-05:00  10.026022  10.067341   9.988050  10.026022   
1       2020-02-29 00:00:00-05:00  10.072952  10.121561  10.048664  10.072952   
2       2020-03-31 00:00:00-04:00   9.888804  10.158980   9.438682   9.888804   
3       2020-04-30 00:00:00-04:00   9.874362   9.930795   9.780902   9.874362   
4       2020-05-31 00:00:00-04:00   9.941533  10.044487   9.860566   9.941533   
...                           ...        ...        ...        ...        ...   
157767  2024-04-30 00:00:00-04:00  92.008212  96.554496  88.085345  91.714539   
157768  2024-05-31 00:00:00-04:00  91.709609  95.153490  85.607545  91.643648   
157769  2024-06-30 00:00:00-04:00  83.943377  88.361752  82.004362  83.800497   
157770  2024-07-31 00:00:00-04:00  84.836553  86.653945  82.024105  84.796622   
157771  2024-08-31 00:00:00-04:00  81.154999  84.650002  76.980003  80.964999   

          Volume Ticker    

In [19]:
combined_df.to_parquet('/Users/paigeblackstone/Desktop/Portfolio29/Portfolio29/Investment_Management/datapull_YF_081224/congress_stock_data.parquet')

calculating average returns over time - need to crosstab data to have the columns represent tickers to use this method - otherwise calculate returns by ticker by month using close price
average trading days in a year: 252. use this number to annualize the returns multiplying them by the trading days in a year